This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_predicted_vs_true(train_quad, test_quad, outdir, label):
    """
    Method to create a parity plot (predicted vs. true values)

    Args:

        train_quad: (tuple), tuple containing 4 numpy arrays: true training y data, predicted training y data,

        training metric data, and groups used in training

        test_quad: (tuple), tuple containing 4 numpy arrays: true test y data, predicted test y data,

        testing metric data, and groups used in testing

        outdir: (str), path to save plots to

        label: (str), label used for axis labeling

    Returns:

        None

    """
    filenames = list()
    y_train_true, y_train_pred, train_metrics, train_groups = train_quad
    y_test_true, y_test_pred, test_metrics, test_groups = test_quad

    # make diagonal line from absolute min to absolute max of any data point
    # using round because Ryan did - but won't that ruin small numbers??? TODO this
    max1 = max(y_train_true.max(), y_train_pred.max(),
               y_test_true.max(), y_test_pred.max())
    min1 = min(y_train_true.min(), y_train_pred.min(),
               y_test_true.min(), y_test_pred.min())
    max1 = round(float(max1), rounder(max1-min1))
    min1 = round(float(min1), rounder(max1-min1))
    for y_true, y_pred, stats, groups, title_addon in \
            (train_quad+('train',), test_quad+('test',)):

        # make fig and ax, use x_align when placing text so things don't overlap
        x_align=0.64
        fig, ax = make_fig_ax(x_align=x_align)

        # set tick labels
        # notice that we use the same max and min for all three. Don't
        # calculate those inside the loop, because all the should be on the same scale and axis
        _set_tick_labels(ax, max1, min1)

        # plot diagonal line
        ax.plot([min1, max1], [min1, max1], 'k--', lw=2, zorder=1)

        # do the actual plotting
        if groups is None:
            ax.scatter(y_true, y_pred, color='blue', edgecolors='black', s=100, zorder=2, alpha=0.7)
        else:
            handles = dict()
            unique_groups = np.unique(np.concatenate((train_groups, test_groups), axis=0))
            log.debug(' '*12 + 'unique groups: ' +str(list(unique_groups)))
            colors = ['blue', 'red', 'green', 'purple', 'orange', 'black']
            markers = ['o', 'v', '^', 's', 'p', 'h', 'D', '*', 'X', '<', '>', 'P']
            colorcount = markercount = 0
            for groupcount, group in enumerate(unique_groups):
                mask = groups == group
                log.debug(' '*12 + f'{group} group_percent = {np.count_nonzero(mask) / len(groups)}')
                handles[group] = ax.scatter(y_true[mask], y_pred[mask], label=group, color=colors[colorcount],
                                            marker=markers[markercount], s=100, alpha=0.7)
                colorcount += 1
                if colorcount % len(colors) == 0:
                    markercount += 1
                    colorcount = 0
            ax.legend(handles.values(), handles.keys(), loc='lower right', fontsize=12)

        # set axis labels
        ax.set_xlabel('True '+label, fontsize=16)
        ax.set_ylabel('Predicted '+label, fontsize=16)

        plot_stats(fig, stats, x_align=x_align, y_align=0.90)

        filename = 'predicted_vs_true_'+ title_addon + '.png'
        filenames.append(filename)
        fig.savefig(join(outdir, filename), dpi=DPI, bbox_inches='tight')

    return filenames


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
train_quad = (array([177.28302038, 150.37102595, 202.0924573 , ...,  47.20790294,
        23.809066  ,   0.        ]), array([230.10561574, 159.87648933, 139.34187537, ...,  63.23618699,
        51.85409151,   3.31492776]), OrderedDict([('R2', 0.7730157396837888), ('root_mean_squared_error', 45.63106199760409), ('mean_absolute_error', 31.173207165583968), ('rmse_over_stdev', 0.4811235904431078)]), None)
test_quad = (array([2.49549443e+02, 2.41537703e+02, 2.96501210e+02, 1.93721694e+02,
       2.09634413e+02, 3.40489474e+02, 1.56515230e+02, 9.45736257e+01,
       9.53431111e+01, 2.41991216e+02, 1.85182335e+02, 1.45936466e+02,
       1.49589711e+02, 1.23826343e+02, 1.17557012e+02, 1.01679243e+02,
       8.91149867e+01, 8.94641218e+01, 1.81755922e+02, 0.00000000e+00,
       1.91511615e+02, 1.95159840e+02, 2.00504538e+02, 1.25776102e+02,
       3.50019597e+02, 3.06666897e+02, 3.19706165e+02, 3.81107807e+02,
       8.44124962e+01, 2.09882629e+02, 7.06664565e+01, 8.09403347e+01,
       7.45855168e+01, 1.87427677e+02, 1.87519625e+02, 1.31350152e+02,
       7.02166732e+01, 0.00000000e+00, 3.87490899e+01, 2.41842340e+01,
       2.48926087e+01, 0.00000000e+00, 7.66050637e+01, 1.83350687e+02,
       1.71608093e+02, 1.20214816e+02, 9.85115662e+01, 2.54265050e+02,
       8.97212986e+01, 1.06616452e+02, 7.19005615e+01, 8.81149314e+01,
       1.57715585e+02, 2.74349274e+02, 1.03801111e+02, 2.78307782e+02,
       8.21644400e+00, 6.33627358e+01, 0.00000000e+00, 9.24561079e+01,
       4.16012879e+01, 5.90096117e+00, 1.29458568e+01, 2.73135364e+01,
       0.00000000e+00, 3.57160729e+02, 2.91632073e+01, 0.00000000e+00,
       1.16763996e+00, 5.38289790e+01, 0.00000000e+00, 3.11290486e+01,
       1.77633032e+02, 1.77441804e+02, 2.36699859e+02, 8.81834984e+01,
       1.45197832e+02, 2.02689680e+02, 3.77502009e+02, 7.51559558e+01,
       7.08357719e+01, 7.12267570e+01, 3.93315709e+01, 4.89115430e+01,
       2.58389489e+01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.18479708e+01, 1.65750556e+01, 0.00000000e+00,
       0.00000000e+00, 4.34882312e+01, 3.21439514e+01, 3.92412752e+01,
       4.31252154e+01, 0.00000000e+00, 4.42073704e-01, 3.92264457e+01,
       1.62308582e+02, 2.10808929e+02, 1.57374363e+02, 0.00000000e+00,
       1.55792869e+02, 9.57636772e+01, 1.00457045e+02, 1.40944441e+01,
       8.73607131e+01, 3.02287500e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 5.05625634e+00, 8.97062587e+00,
       5.53782777e+01, 6.21108106e+01, 4.07973166e+01, 5.08263675e+01,
       2.39414173e+01, 1.17125778e+02, 2.41335092e+01, 1.22038739e+02,
       5.38520339e+01, 2.18803383e+01, 4.76192799e+01, 2.61695184e+01,
       1.33902387e+01, 2.36626028e+01, 5.74995003e+01, 4.85711869e+01,
       7.30016409e+01, 1.05693165e+02, 1.44194572e+02, 9.39602340e+01,
       8.65356568e+01, 6.52948104e+01, 1.00859781e+02, 8.51942082e+01,
       0.00000000e+00, 0.00000000e+00, 9.85807552e+01, 0.00000000e+00,
       3.50982856e+00, 1.87391046e+02, 0.00000000e+00, 4.70862505e+01,
       2.76028721e+02, 2.57075963e+02, 4.45414483e+01, 1.85524542e+02,
       5.20286768e+01, 2.94415703e+02, 1.39763133e+02, 1.97246257e+02,
       4.21689745e+01, 1.26777344e+02, 5.08392287e+01, 4.78013760e+01,
       3.81465155e+01, 2.23740347e+01, 4.22355880e+01, 4.08070147e+01,
       0.00000000e+00, 2.80575506e+01, 9.53324888e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.40097275e+00, 1.21655324e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 5.88853197e+01, 7.20108022e+01, 5.85465181e+01,
       0.00000000e+00, 9.00043474e+01, 6.66463830e+01, 5.46661575e+01,
       5.87368740e+01, 9.70888906e+01, 5.02984410e+01, 5.13090164e+01,
       5.33902792e+01, 6.08628852e+01, 3.95429854e+01, 3.40111139e+01,
       4.12956968e+01, 0.00000000e+00, 1.89559087e+01, 1.38758173e+02,
       1.50562981e+02, 1.24729865e+02, 1.28386129e+02, 1.95442993e+02,
       0.00000000e+00, 2.64912691e+01, 1.02395743e+02, 1.40515010e+02,
       4.27492462e+01, 2.36410205e+01, 1.15904985e+01, 4.23440436e+01,
       5.50100622e+01, 1.40011496e+01, 5.76088495e+01, 9.54966739e+01,
       2.32281891e+01, 8.63070905e+00, 6.89012756e+00, 0.00000000e+00,
       2.32118071e+01, 0.00000000e+00, 5.40607889e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.33030325e+02,
       1.59117705e+02, 1.79412889e+02, 1.82075421e+02, 2.50252038e+02,
       2.09019845e+02, 2.15604850e+02, 1.58037965e+02, 3.61579777e+02,
       3.66503081e+02, 1.34673117e+02, 1.10139318e+02, 9.77903794e+01,
       1.42295818e+02, 2.43428371e+02, 2.24622871e+02, 1.89824630e+02,
       2.20003381e+02, 9.81125625e+01, 1.01768108e+02, 1.35390845e+02,
       2.51196620e+02, 1.54419624e+02, 2.92536084e+02, 3.51817502e+02,
       2.36338029e+02, 3.57145351e+02, 2.62896301e+02, 2.26507616e+02,
       2.06364164e+02, 2.21055904e+02, 3.40306363e+02, 2.55478983e+02,
       1.99002346e+02, 7.64960376e+01, 1.11026328e+02, 1.80660958e+02,
       7.08220872e+01, 8.23163755e+01, 2.10107835e+02, 1.95672754e+02,
       8.86267599e+01, 2.80854292e+02, 1.03456135e+02, 1.93969856e+02,
       8.84773614e+01, 1.35408933e+02, 1.08425980e+02, 1.21127399e+02,
       3.13722881e+02, 1.60233988e+02, 2.59216571e+02, 2.38988178e+02,
       3.05414508e+02, 1.20403530e+02, 1.42826192e+02, 7.89063338e+01,
       1.24842572e+02, 6.84009027e+01, 1.70005981e+02, 9.73721500e+01,
       1.18314107e+02, 1.39928045e+02, 1.51898311e+02, 6.02850907e+01,
       5.55045542e+01, 1.58202913e+02, 1.33883367e+02, 1.29155595e+02,
       1.80035168e+02, 8.31507296e+01, 8.68301581e+01, 1.84907725e+02,
       2.38535058e+02, 1.93517827e+02, 2.27717761e+02, 2.28169699e+02,
       2.95582043e+02, 2.04542101e+02, 3.18785354e+02, 2.71906598e+02,
       9.56831956e+02, 1.51155130e+02, 1.28639131e+02, 1.81019902e+02,
       1.10472455e+02, 1.35426489e+02, 1.07655170e+02, 8.10050964e+01,
       7.82152525e+01, 1.41354043e+02, 8.46282081e+01, 1.94691060e+02,
       9.45208382e+01, 8.28280997e+01, 9.43913417e+01, 1.30568582e+02,
       1.78159873e+02, 1.06913117e+02, 9.64735725e+01, 3.60066863e+01,
       6.28478449e+01, 9.94733566e+00, 3.31481761e+01, 5.18182329e+01,
       3.70668895e+01, 4.43281812e+01, 1.24273117e+02, 4.34380611e+01,
       7.19683460e+01, 4.30159281e+01, 1.33635430e+02, 5.12762345e+01,
       7.75933817e+01, 2.34937875e+02, 9.52060454e+01, 1.46552082e+02,
       1.72754544e+02, 1.15407866e+02, 1.81387575e+02, 1.59787224e+02,
       1.71603969e+02, 3.32804978e+01, 2.71319339e+02, 1.02775420e+02,
       1.22316043e+02, 1.26798490e+02, 3.97239965e+02, 1.11066722e+02,
       1.17140514e+02, 1.13501499e+02, 1.36808521e+02, 1.35369317e+02,
       8.98167378e+01, 9.07487562e+01, 1.34354983e+02, 4.21335070e+01,
       2.86171985e+01, 1.18947119e+02, 2.52930530e+01, 1.51299348e+01,
       1.91667256e+01, 9.50490064e+01, 5.28734187e+01, 1.40488835e+02,
       1.11369551e+02, 1.01020674e+02, 3.50703870e+01, 9.55308332e+01,
       8.29134753e+01, 0.00000000e+00, 4.92192870e+01, 0.00000000e+00,
       7.32347672e+01, 5.19157130e+01, 1.01755733e+02, 1.60849161e+02,
       6.39648556e+01, 2.13539239e+01, 2.84812922e+01, 2.83147240e+01,
       6.11195316e+01, 2.86637589e+01, 0.00000000e+00, 2.67854426e+02,
       1.39062159e+02, 7.51456208e+01, 7.68437187e+01, 2.21768007e+02,
       6.89710696e+01, 6.23324679e+01, 4.07002198e+01, 5.22740325e+01,
       3.60966810e+01, 4.69507504e+01, 5.50886160e+01, 5.25504455e+01,
       4.39454900e+01, 4.60103452e+01, 3.83815528e+01, 0.00000000e+00,
       1.03731415e+02, 6.76609259e+01, 3.15229082e+01, 8.78388717e+01,
       3.48683785e+01, 4.11379073e+01, 3.74352793e+01, 3.12618672e+01,
       5.78409146e+01, 2.45125079e+01, 4.67667976e+01, 4.50623816e+01,
       2.28817925e+01, 8.97135102e+01, 1.25245007e+02, 9.90236861e+01,
       7.83039861e+01, 1.58851259e+00, 0.00000000e+00, 0.00000000e+00,
       3.65545469e+01]), array([ 3.03159748e+02,  1.58753014e+02,  1.61863742e+02,  1.46563870e+02,
        1.47830990e+02,  2.38424780e+02,  1.14465899e+02,  1.05984182e+02,
        9.22999265e+01,  2.30167073e+02,  1.53353810e+02,  1.36440114e+02,
        1.58347120e+02,  1.45489360e+02,  1.60419203e+02,  9.28851027e+01,
        8.84113780e+01,  8.58736739e+01,  1.25388956e+02,  1.58690375e+02,
        2.88318794e+02,  1.86071087e+02,  2.12514001e+02,  1.51404142e+02,
        2.32235490e+02,  2.65684039e+02,  2.58397502e+02,  3.04328033e+02,
        1.32559574e+02,  1.02185747e+02,  8.95292978e+01,  8.44107540e+01,
        9.24028441e+01,  1.63498753e+02,  1.44099040e+02,  2.66082788e+01,
        3.74632248e+01,  3.66500393e+01,  1.82887949e+01,  5.66160646e+01,
        4.63523907e+01,  1.40362548e+01,  1.52104819e+02,  1.17585025e+02,
        1.27107244e+02,  6.17356283e+01,  7.71058600e+01,  1.11722327e+02,
        8.84591430e+01,  8.57684164e+01,  7.91911609e+01,  6.69228029e+01,
        9.36687626e+01,  2.91435225e+02,  1.33626483e+02,  1.71167075e+02,
        3.42637055e+00,  2.24581769e+01,  1.09685012e+02,  9.49551017e+01,
        1.46324662e+02,  9.66114634e+01,  1.06600220e+02,  7.97946151e+01,
        8.36704241e+01,  2.86694331e+02,  2.64181577e+01,  3.90977486e+01,
        1.51776838e+01,  3.41118252e+01,  3.43059874e+01,  2.80343732e+01,
        1.36937563e+02,  7.74098757e+01,  7.17400704e+01,  7.75888826e+01,
        1.54988380e+02,  1.64899331e+02,  2.84099375e+02,  8.34232790e+01,
        8.63568329e+01,  1.07219353e+02,  5.66811373e+01,  7.76923567e+01,
        6.07912957e+01,  2.48386342e+00,  4.68565966e+01,  2.69961041e+01,
        1.87906632e+01,  1.41636851e+01,  2.79019860e+01,  5.37919963e+01,
        1.93022628e+01,  4.68597954e+01,  6.40366506e+01,  3.62750496e+01,
        4.47391885e+01,  3.28619656e+01,  4.76988821e+01,  6.32601893e+01,
        2.95779728e+01,  2.61258491e+01,  6.71365651e+01,  3.16796372e+01,
        1.81396797e+02,  2.61703537e+02,  1.04079692e+02,  9.02889213e+00,
        1.70869338e+02,  8.80263629e+01,  9.65755463e+01,  5.10008152e+01,
        8.16348516e+01,  1.99784243e+01,  1.42337736e+01,  6.06867419e+01,
        1.96981534e+01,  3.86727616e+01,  3.42366036e+01,  2.49674266e+01,
        3.53318095e+01,  5.02790743e+01,  2.88793333e+01,  3.57907186e+01,
        6.02731982e+01,  3.09765500e+01,  2.28945837e+01,  7.66561158e+01,
        1.20017108e+01,  1.97443072e+01,  2.59808646e+01,  9.78637888e+00,
        2.75492580e+01,  2.07495169e+01,  5.61284460e+01,  2.40387411e+01,
        6.45702426e+01,  1.21620098e+02,  1.35293863e+02,  7.56071942e+01,
        8.30046742e+01,  9.76904238e+01,  1.16837026e+02,  6.07089480e+01,
        5.48933818e+01,  7.01149545e+01,  7.21203291e+01,  5.46216178e+01,
        6.45966865e+01,  1.27831448e+02,  9.17485728e+01,  8.18609244e+01,
        2.09908372e+02,  2.96199890e+02,  1.28456374e+02,  1.04649873e+02,
        3.21537255e+01,  2.47079133e+02,  5.79976249e+01,  8.03488340e+01,
        2.53803175e+01,  2.97293074e+01,  1.37405851e+02,  6.30196282e+01,
        7.62751294e+01,  4.81109542e+01,  5.50396967e+01,  3.79502692e+01,
       -3.08062321e+00,  6.64401074e+01,  8.87245774e+01,  4.78999552e-01,
        1.72438030e+00,  1.13417295e+01,  1.03630382e+01,  3.71024996e+01,
        7.38350700e+00,  1.27720535e+01,  2.13362155e+01,  3.06488355e+01,
        2.90196930e+01,  6.89683940e+01,  6.77238074e+01,  6.76919476e+01,
       -3.56581390e+00,  8.19788598e+01,  1.36826195e+02,  7.17577747e+01,
        7.63601575e+01,  1.22814441e+02,  7.02343652e+01,  6.87955026e+01,
        6.26483402e+01,  5.81034512e+01,  5.14635579e+01,  6.79601017e+01,
        1.43382569e+01,  2.48191701e+01,  2.67522728e+01,  1.16716266e+02,
        1.37800044e+02,  1.10221896e+02,  1.11209413e+02,  2.41600896e+02,
        1.16815601e+01,  3.04634872e+01,  8.94342781e+01,  8.25079276e+01,
        2.49100703e+01,  2.88333826e+01,  3.17600423e+01,  4.28057893e+01,
        5.06286494e+01,  8.97913183e+00,  2.28824245e+01,  1.87852043e+01,
        2.42358508e+01,  2.32211756e+01,  3.66796895e+01,  4.83949389e+01,
        3.83495893e+01,  4.30895555e+01,  1.41416620e+01,  1.18857819e+01,
        1.24446862e+01,  4.88220246e+00,  2.76251744e+01,  2.18126040e+02,
        2.14537451e+02,  1.47034226e+02,  1.61755513e+02,  3.00504310e+02,
        2.64684455e+02,  1.86832545e+02,  1.18931640e+02,  2.99167335e+02,
        2.93130617e+02,  2.12151608e+02,  1.30485740e+02,  1.22666694e+02,
        1.50884324e+02,  2.94701218e+02,  1.73497242e+02,  1.85489116e+02,
        2.20939826e+02,  1.00771281e+02,  6.50460772e+01,  1.21514135e+02,
        1.42169641e+02,  1.33163785e+02,  1.41367818e+02,  3.02555391e+02,
        2.49579044e+02,  3.00652105e+02,  2.52662010e+02,  1.92564174e+02,
        2.29824142e+02,  2.49745413e+02,  2.68600569e+02,  2.94303567e+02,
        1.70569055e+02,  8.04475961e+01,  8.10818924e+01,  2.09465108e+02,
        8.58913112e+01,  8.05620570e+01,  8.40260721e+01,  1.50102127e+02,
        1.29527296e+02,  1.97001324e+02,  8.48468955e+01,  9.97070428e+01,
        7.82521848e+01,  1.20369988e+02,  1.16593706e+02,  1.16685655e+02,
        1.91736800e+02,  1.30099565e+02,  2.42620367e+02,  2.04283729e+02,
        2.54722321e+02,  1.97261859e+02,  1.65102562e+02,  8.09480190e+01,
        1.87994719e+02,  8.11466338e+01,  8.27064310e+01,  1.43207188e+02,
        1.61878322e+02,  1.85674151e+02,  1.33155263e+02,  9.54649996e+01,
        9.50925593e+01,  9.96672378e+01,  1.30787499e+02,  1.15001376e+02,
        1.51933514e+02,  1.34619328e+02,  1.14605368e+02,  1.41181239e+02,
        1.67340860e+02,  2.23900098e+02,  2.71859303e+02,  2.78386855e+02,
        2.73498856e+02,  2.21542708e+02,  2.35367199e+02,  5.65201117e+02,
        5.26629598e+02,  1.70725244e+02,  1.26552071e+02,  1.23800422e+02,
        1.40177672e+02,  1.38248495e+02,  1.23677888e+02,  1.17031971e+02,
        1.23468020e+02,  1.32255439e+02,  1.19569247e+02,  1.24517058e+02,
        1.28660181e+02,  1.21193395e+02,  1.28609248e+02,  1.36908432e+02,
        1.58212998e+02,  1.35072698e+02,  1.21998075e+02,  5.68116515e+01,
        6.82196669e+01,  3.56093987e+01,  3.40465154e+01,  3.66046948e+01,
        1.07302560e+02,  6.31360484e+01,  8.22470293e+01,  1.06450522e+02,
        1.24519712e+02,  6.62375953e+01,  7.99720481e+01,  3.64500827e+01,
        4.15721746e+01,  1.65443807e+02,  1.70422458e+02,  1.20621314e+02,
        1.92278454e+02,  1.26255462e+02,  2.41544897e+02,  1.66100246e+02,
        1.67514673e+02,  9.90022284e+01,  2.95223989e+02,  1.07883136e+02,
        2.21389210e+02,  1.46025656e+02,  3.05619331e+02,  1.94415565e+02,
        1.08745153e+02,  1.32081141e+02,  1.35947430e+02,  1.05571298e+02,
        7.54966882e+01,  1.28895137e+02,  1.45665738e+02,  6.74316506e+01,
        3.84239848e+01,  8.86326533e+01,  3.12299597e+01,  4.67902683e+01,
        3.14006278e+01,  5.45705412e+01,  6.87886552e+01,  1.09037981e+02,
        9.98663550e+01,  5.79887337e+01,  1.20144444e+02,  8.26858358e+01,
        1.06484752e+02,  5.90290095e+00,  4.40969168e+01, -5.29110662e+00,
        5.87285907e+01,  6.97060349e+01,  3.94895469e+01,  7.55768007e+01,
        5.11267832e+01,  1.09714657e-01,  6.28281682e+00,  7.82421941e+00,
        3.29206176e+01,  4.19193210e+01,  3.22367780e+01,  1.62101979e+02,
        9.63158271e+01,  7.31987548e+01,  4.60207897e+01,  1.50780862e+02,
        5.54851227e+01,  3.89856316e+01,  7.09309584e+01,  5.33029687e+01,
        6.98655538e+01,  3.20872699e+01,  4.25306092e+01,  6.48124485e+01,
        2.49396366e+01,  2.25754925e+01,  2.82548385e+01,  2.64435060e+01,
        7.48032633e+01,  7.94904489e+01,  6.82836506e+01,  9.32236419e+01,
        1.68042441e+01,  2.72345857e+01,  1.70987464e+01,  2.08552026e+01,
        6.16259664e+01,  3.11077614e+01,  3.13688413e+01,  2.57544075e+01,
        3.12198439e+01,  6.77168680e+01,  7.87609304e+01,  5.64552297e+01,
        3.21194210e+01,  5.92410628e+01,  4.60881420e+01,  2.96618445e+01,
        2.40474781e+01]), OrderedDict([('R2', 0.7235687061630367), ('root_mean_squared_error', 51.423268881045935), ('mean_absolute_error', 35.271059094292504), ('rmse_over_stdev', 0.5421953089250741)]), None)
outdir = '/home/nerve/Desktop/skunkworks/lane_schultz/learning_curves/learning/StandardScaler/SequentialFeatureSelector/KernelRidge/RepeatedKFold/split_6'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_paths = plot_predicted_vs_true(train_triple, test_triple, outdir)
for plot_path in plot_paths:
    display(Image(filename=plot_path))
